<a href="https://colab.research.google.com/github/sanjit1995/MothersDayML_HackerEarth/blob/master/MothersDayML_HE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import numpy as np

read data and separate

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/MothersDayML_HE/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/MothersDayML_HE/test.csv")

In [0]:
train['sentiment_class'].value_counts()

In [0]:
pd.crosstab(index = train['sentiment_class'], columns="Total count")

In [0]:
train.isnull().sum()

In [0]:
x_data = train.loc[:, ['original_text']]
y_data = train['sentiment_class']

In [0]:
test_data = test.loc[:, ['original_text']]

In [5]:
x_data.shape

(3235, 1)

In [6]:
y_data.shape

(3235,)

In [25]:
test_data.shape

(1387, 1)

remove web urls from the texts and special characters except # and '

for train

In [0]:
x_data['new_text'] = x_data['original_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'pic.\S+', '', regex=True).replace(r'photos.app\S+', '', regex=True).replace(r'twitter.com\S+', '', regex=True)

In [0]:
x_data['new_text'] = x_data['new_text'].map(lambda x: re.sub(r'[^A-Za-z\s\#\']+', '', x))

for test

In [0]:
test_data['new_text'] = test_data['original_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'pic.\S+', '', regex=True).replace(r'photos.app\S+', '', regex=True).replace(r'twitter.com\S+', '', regex=True)

In [0]:
test_data['new_text'] = test_data['new_text'].map(lambda x: re.sub(r'[^A-Za-z\s\#\']+', '', x))

In [0]:
pd.set_option('display.max_colwidth', None)
x_data.head(3)

In [0]:
test_data.head(3)

fit_transform the train_data

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    min_df=1,  # min count for relevant vocabulary
    max_features=2500,  # maximum number of features
    strip_accents='unicode',  # replace all accented unicode char 
    # by their corresponding  ASCII char
    analyzer='word',  # features made of words
    token_pattern=r'\w{1,}',  # tokenize only words of 4+ chars
    ngram_range=(1, 1),  # features made of a single tokens
    use_idf=True,  # enable inverse-document-frequency reweighting
    smooth_idf=True,  # prevents zero division for unseen words
    sublinear_tf=False)

tfidf_df = tfidf_vectorizer.fit_transform(x_data['new_text'])

just transform the test_data according to train_data

In [0]:
x_test = tfidf_vectorizer.transform(test_data['new_text'])

In [0]:
tfidf_vectorizer.get_feature_names()

In [0]:
tfidf_df

In [0]:
x_train = tfidf_df.copy()
y_train = y_data.copy()

do oversampling

In [0]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(sampling_strategy='not majority', random_state=42)
X_res,Y_res=smk.fit_sample(x_train,y_train)

In [60]:
X_res.shape

(5055, 2500)

In [61]:
Y_res.shape

(5055,)

In [39]:
x_train.shape

(3235, 2500)

In [23]:
y_train.shape

(3235,)

In [40]:
x_test.shape

(1387, 2500)

Training part

using LogisticRegression

In [63]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
model = LogisticRegression(max_iter=200)
model.fit(X_res, Y_res)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

using SGDClassifier

In [69]:
model = SGDClassifier()
model.fit(X_res, Y_res)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

Using NaiveBayes Classifier

In [0]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB() 
model.fit(X_res.toarray(), Y_res) 

In [0]:
y_pred = model.predict(x_test.toarray())
y_pred

In [71]:
len(y_pred)

1387

In [80]:
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{-1: 90, 0: 1190, 1: 107}

In [0]:
pred = pd.DataFrame(y_pred)
sub_df = test
sub_df = sub_df.drop(['original_text','lang','retweet_count','original_author'], axis=1)
datasets = pd.concat([sub_df[['id']],pred], axis=1)
datasets.columns = ['id','sentiment_class']
datasets.sentiment_class = datasets.sentiment_class.astype(int)
datasets.to_csv('/content/my_submission4.csv', index=False)

In [0]:
datasets

In [14]:
indices = np.argsort(tfidf_vectorizer.idf_)[::-1]
features = tfidf_vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
top_features

['naturephotography', 'marina']

In [0]:
feature_array = np.array(tfidf_vectorizer.get_feature_names())
tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
top_n